In [1]:
import torch
import numpy as np
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from transformers import AlbertTokenizer, AlbertModel
from torch import cuda
import pandas as pd
import numpy as np
import torch
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
from torch.optim import Adam, AdamW
from torch.nn import CrossEntropyLoss
import logging
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, CosineAnnealingWarmRestarts

device = 'cuda' if cuda.is_available() else 'cpu'


/home/kaie/miniconda3/envs/discord/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 2)
        self.soft = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.soft(output)
        return output

class AlbertClass(torch.nn.Module):
    def __init__(self):
        super(AlbertClass, self).__init__()
        self.l1 = AlbertModel.from_pretrained("albert-large-v2")
        self.pre_classifier = torch.nn.Linear(1024 , 512 )
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(512 , 2)
        self.soft = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # print(output_1)
        hidden_state = output_1[0]
        # print(hidden_state)
        pooler = hidden_state[:, 0]
        # print(pooler)
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.soft(output)
        return output

In [3]:
# model = AlbertClass()
model = RobertaClass()
# model = torch.load('models/roberta_loss_0.3265.pth')
model.to(device)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
# tokenizer = AlbertTokenizer.from_pretrained('albert-xlarge-v2', truncation=True, do_lower_case=True)


/home/kaie/miniconda3/envs/discord/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

# input = tokenizer(text = "Testimony two, or the testimony of the security officer. Too formal language, the time is off (the event happened at 11 pm), the suspect was on a different street (Kings Street), didn't even try to stop the suspect." , text_pair="The officer's testimony, needless to say, was far too formal, and it incorrectly documented the wrong time, specifically 11 pm, also inaccurately pinpointing the wrong street, and, regrettably, there was no attempt made to chase the suspect.", return_tensors='pt', return_token_type_ids=True )
# print(input)
# token_type_ids = input['token_type_ids'].to(device)

# mask = input['attention_mask'].to(device)
# tokens = input['input_ids'].to(device)
# print(tokens.shape)
# # print(tokens)
# model.eval()

# print(model(tokens, mask, token_type_ids)[0][0])
# print(model(tokens, mask, token_type_ids)[0][1])

In [5]:

df = pd.read_excel("Is it real_ (Responses).xlsx", sheet_name="Form Responses 1")

In [6]:
Promopt = df['Correct']
X = df['Problem 1']
Y = df['Ground Truth'].to_list()
z = [[a, b] for a, b in zip(Promopt, X)]


In [7]:


X_train, X_test, y_train, y_test = train_test_split(z, Y , test_size=0.2)
# X_train = z
# y_train = Y

print(len(X_train))

print(len(X_test))

2079
520


In [8]:
class CustomDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        # print(f"Accessing index: {index}")
        # print(f"Total texts: {len(self.texts)}, Total targets: {len(self.targets)}")

        # if index >= len(self.texts) or index >= len(self.targets):
        #     raise IndexError(f"Index {index} out of bounds for dataset with {len(self.texts)} texts and {len(self.targets)} targets")

        text = self.texts[index][0].strip().lower()
        answer = self.texts[index][1].strip().lower()
        target = self.targets[index]
        encoding = self.tokenizer.encode_plus(
            text=text,
            text_pair=answer,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }


In [9]:
max_len = 512
batch_size = 13
epochs = 10000
train_dataset = CustomDataset(X_train, y_train, tokenizer, max_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = CustomDataset(X_test, y_test, tokenizer, max_len)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

optimizer = AdamW(model.parameters(), lr=5e-6, weight_decay=0.01)
print(len(train_loader))
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=5 * len(train_loader))
criterion = CrossEntropyLoss()

160


In [10]:
lowest_loss = 0.35

In [11]:
# Training loop

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_val_loss = 0
    correct_val_predictions = 0

    for batch in tqdm(train_loader, desc=f"Training: Epoch {epoch + 1}/{epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        targets = batch['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask, token_type_ids)
        loss = criterion(outputs, targets)
        prediction = torch.argmax(outputs, dim=1)
        
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct_predictions += (prediction == targets).sum().item()
        scheduler.step()

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / len(train_loader.dataset)
    lr = scheduler.get_last_lr()[0]
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.6f}, Accuracy: {accuracy:.6f}, learning rate: {lr:.10f}")
    
    model.eval()
    for batch in tqdm(test_loader, desc=f"Validation: Epoch {epoch + 1}/{epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        targets = batch['targets'].to(device)

        outputs = model(input_ids, attention_mask, token_type_ids)
        loss = criterion(outputs, targets)
        prediction = torch.argmax(outputs, dim=1)
        total_val_loss += loss.item()
        correct_val_predictions += (prediction == targets).sum().item()     
    avg_loss = total_val_loss / len(test_loader)
    accuracy = correct_val_predictions / len(test_loader.dataset)
    print(f"Validation {epoch + 1}, Loss: {avg_loss:.6f}, Accuracy: {accuracy:.6f}")
    if lowest_loss > avg_loss:
        model_weights_path = f"./models/roberta_loss_{avg_loss:.6f}.pth"
        tokenizer_path = "./robertatokenizer/"
        torch.save(model, model_weights_path)
        tokenizer.save_pretrained(tokenizer_path)
        lowest_loss = avg_loss
    



Training: Epoch 1/10000: 100%|██████████| 160/160 [00:47<00:00,  3.34it/s]


Epoch 1, Loss: 0.657245, Accuracy: 0.613276, learning rate: 0.0000045225


Validation: Epoch 1/10000: 100%|██████████| 520/520 [00:05<00:00, 96.36it/s]


Validation 1, Loss: 0.586139, Accuracy: 0.684615


Training: Epoch 2/10000: 100%|██████████| 160/160 [00:54<00:00,  2.94it/s]


...

Epoch 456, Loss: 0.317315, Accuracy: 0.995671, learning rate: 0.0000045225


Validation: Epoch 456/10000: 100%|██████████| 520/520 [00:05<00:00, 87.13it/s]


Validation 456, Loss: 0.331692, Accuracy: 0.980769


Training: Epoch 457/10000:  36%|███▋      | 58/160 [00:19<00:33,  3.04it/s]


KeyboardInterrupt: 

In [ ]:
# model_weights_path = "./models/albert.pth"
# tokenizer_path = "./alberttokenizer/"
# torch.save(model, model_weights_path)
# tokenizer.save_pretrained(tokenizer_path)
# model_weights_path = "./models/roberta.pth"
# tokenizer_path = "./robertatokenizer/"
# torch.save(model, model_weights_path)
# tokenizer.save_pretrained(tokenizer_path)